[RAG 처리]

1. 문서의 내용을 읽는다.
2. 문서를 쪼갠디
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 답변 생성이 오래걸림

3. 임베딩 -> 백터DB에 저장
4. 질문이 있을때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)


In [13]:
document_list[52]

Document(metadata={'source': './tax_with_markdown.docx', 'text': '제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한

In [6]:
# 임베딩

from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model='text-embedding-3-large')


In [ ]:
# # 인메모리 백터DB

# from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding)

In [7]:
# Pinecone - cloud vectorDB 

import os 
import time
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name = "tax-makrdown-index"

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [ ]:
# query = '연봉 5천만원인 직장인의 소득세는 얼마인가요? 대략적인 소득세가 얼마인지만 알려주세요.'
# retrieved_docs = database.similarity_search(query=query)

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')


In [9]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

In [10]:
# query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
query = '연봉 5천만원의 소득세를 구체적으로 계산하고, 세율 및 공제 내용을 포함해 설명해 주세요.'

In [11]:
from langchain_classic.chains import RetrievalQA


retriever=database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt}
)



In [12]:
retriever = database.as_retriever()
retriever.invoke(query)

[Document(id='02209d86-f0ca-4bd2-ada0-9b547762df04', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

In [13]:

ai_message = qa_chain({'query':query})

C:\Users\stron\AppData\Local\Temp\ipykernel_19824\4015335110.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  ai_message = qa_chain({'query':query})


In [25]:
(ai_message)

{'query': '연봉 5천만원의 소득세를 구체적으로 계산하고, 세율 및 공제 내용을 포함해 설명해 주세요.',
 'result': '연봉 5천만 원의 소득세를 계산하려면, 1,400만 원 초과 5,000만 원 이하에 해당하는 세율을 적용합니다. 계산은 84만 원에 1,400만 원을 초과하는 금액(5,000만 원 - 1,400만 원 = 3,600만 원)의 15%를 더합니다. 따라서, 종합소득세는 84만 원 + 3,600만 원 × 0.15 = 624만 원입니다.'}

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary=["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변결할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [15]:
new_question = dictionary_chain.invoke({"question":query})

In [32]:
(new_question)

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [16]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [17]:
ai_response = tax_chain.invoke({"question":query})

In [18]:
(ai_response)

{'query': '연봉 5천만원의 소득세를 구체적으로 계산하고, 세율 및 공제 내용을 포함해 설명해 주세요.',
 'result': '연봉 5천만원의 소득세는 1,400만원 초과 5,000만원 이하 구간에 해당하므로, 기본세액 84만원에 1,400만원을 초과한 금액(3,600만원)의 15%를 추가하여 계산됩니다. 따라서, 소득세는 84만원 + (3,600만원 × 0.15) = 624만원입니다. 세액 공제를 적용하기 전의 산출세액입니다.'}